ArcGIS has a built in Python interpreter (the only saving grace of that piece of software).  This code is copy/pasted into ArcGIS to extract rasters for topology and fuel load matching the fire perimeters.

In [7]:
import pickle
import numpy as np
import pandas as pd

df = pickle.load(open('2012_fire_bounds.pkl', 'rb'))
df.head()

,fire_id,left,right,top,bottom,x,y,a
2,ID-SCF-G4A0,-1635840.0,-1390080.0,2711040.0,2257920.0,8192.0,15104.0,1.113588e+11
0,WA-OWF-G70S,-1912320.0,-1712640.0,3064320.0,2872320.0,6656.0,6400.0,3.833856e+10
27,VA-VAF-GR0U,1359360.0,1505280.0,1904640.0,1720320.0,4864.0,6144.0,2.689597e+10
7,MN-MNS-G85U,-76800.0,76800.0,2895360.0,2764800.0,5120.0,4352.0,2.005402e+10
64,MT-MCD-G0QM,-867840.0,-760320.0,2603520.0,2488320.0,3584.0,3840.0,1.238630e+10


In [ ]:
import pandas as pd

layers = ['us_slp', 'us_dem', 'us_asp', 'us_130fbfm40']
#'us_130fbfm40'

def get_neighborhood(layer, left, bottom, ncols, nrows):

    offset= 0
    left = left +offset
    bottom = bottom+offset

    a = arcpy.RasterToNumPyArray(
        layer,
        arcpy.Point(left, bottom),
        ncols,
        nrows
    )
    return a

def create_array(layers, name, left, bottom, ncols, nrows):
    data = dict()
    data['fire_id'] = name
    data['ref_point'] = (left, bottom)
    data['shape'] = (ncols, nrows)
    data['type'] = 'landfire'
    
    for layer in layers:
        data[layer]=get_neighborhood(layer, left, bottom, ncols, nrows)
    
    fn = name+ ' topo.pkl'
    outfile = open('topo/'+fn,'wb')
    pickle.dump(data,outfile)
    outfile.close()

In [ ]:
df.sort_values('a', inplace=True)

for row in df.itertuples():
    name = row[1]
    left =row[2]
    bottom=row[5]
    ncols = row[6]
    nrows = row[7]
    
    print(name, row[8])
    create_array(layers, name, left, bottom, ncols, nrows)